In [1]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
!pwd
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../../quant_container/src')
sys.path.append('../src')



from mosaicsmartdata.common import qc_csv_helper
from mosaicsmartdata.core.quote import Quote
import csv, copy

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pickle

try:
    with open('data.pickle', 'rb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        quote_dict = pickle.load(f)
except:
#if True:
    file_path ='../resources/fx/'
    filenames = ['OIS-N156898469.csv','EUR_JPY_GBP_till_SN.csv','EUR_JPY_GBP_from_SW_till_1Y.csv']
    quote_dict_0 = qc_csv_helper.get_ric_quotes(file_path+filenames[0])
    print(len(quote_dict_0), quote_dict_0.keys())
    quote_dict_1 = qc_csv_helper.get_ric_quotes(file_path+filenames[1])
    print(len(quote_dict_1), quote_dict_1.keys())
    quote_dict_2 = qc_csv_helper.get_ric_quotes(file_path+filenames[2])
    print(len(quote_dict_2), quote_dict_2.keys())
    quote_dict = dict(quote_dict_0,**quote_dict_1)
    quote_dict = dict(quote_dict,**quote_dict_2)
    with open('data.pickle', 'wb') as f:
        pickle.dump(quote_dict, f)

In [3]:
x = list(quote_dict.keys())
x.sort()
print(x)

['EUR1M=', 'EUR1Y=', 'EUR3M=', 'EUR6M=', 'EUR=', 'EURON=', 'EURTN=', 'GBP1M=', 'GBP1Y=', 'GBP3M=', 'GBP6M=', 'GBP=', 'GBPON=', 'GBPTN=', 'JPY1M=', 'JPY1Y=', 'JPY3M=', 'JPY6M=', 'JPY=', 'JPYON=', 'JPYTN=', 'USD1MOIS=', 'USD1YOIS=', 'USD3MOIS=', 'USD6MOIS=', 'USDSWOIS=']


In [4]:
# create a trade
from mosaicsmartdata.core.instrument_utils import sym_to_instrument
from mosaicsmartdata.core.trade import Trade, FXForwardTrade

#spot_quotes = [quotes for sym, quotes in quote_dict.items() if len(sym)==4]
my_quote = quote_dict['EUR='][100]
my_instr = sym_to_instrument()(my_quote.sym, my_quote.timestamp.date())
my_instr.notionals = (100, -100*my_quote.bid)
print(my_instr)
print('*****')
my_trade = FXForwardTrade(my_instr, 
                          trade_id = '12345',
                        timestamp = my_quote.timestamp)
print(my_trade)
print('*******')
print(my_quote)

<class 'mosaicsmartdata.core.instrument.FXForward'> :  static = Instrument static singleton facade, notionals = (100, -111.97999999999999), sym = EUR=, venue = None, settle_date = 2017-06-28, maturity_date = 2017-06-28, isDeliverable = True, pip_size = 0.0001, holidayCities = None, date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x00000176C585D358>, ccy = ('EUR', 'USD'), tenor = SPOT
*****
<class 'mosaicsmartdata.core.trade.FXForwardTrade'> :  trade_id = 12345, timestamp = 2017-06-26 00:02:26.995000, leg_no = None, delta = 1.0, sales = None, instrument = <class 'mosaicsmartdata.core.instrument.FXForward'> :  static = Instrument static singleton facade, notionals = (100, -111.97999999999999), sym = EUR=, venue = None, settle_date = 2017-06-28, maturity_date = 2017-06-28, isDeliverable = True, pip_size = 0.0001, holidayCities = None, date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x00000176C585D358>, ccy = ('EUR', 'USD'), tenor = SPO

In [5]:
from aiostreams import run
import aiostreams.operators as op
from mosaicsmartdata.core.markout import MarkoutCalculator
import logging

In [6]:
# run markouts based on simple instrument ID, with no pricing context
# logging.getLogger().setLevel('INFO')
# logging.info('starting command line test...')

# run the markout calc
trades = [my_trade]
quotes = quote_dict['EUR=']
stream = op.merge_sorted([quotes[:1000], trades], lambda x: x.timestamp) \
            | op.flat_map_by_group(lambda x: x.sym, MarkoutCalculator([-10, 0, 10, 100]))\
            > print
        
run(stream)

 dt:-10 final_price:1.1199 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:16.995000 package:None PV:9.999999999998899e-05 price:9.999999999998899e-05 timestamp:2017-06-26 00:02:26.995000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None trade_maturity_date:28/06/2017 traded_px:1.1198 ccy:('EUR', 'USD')
 dt:0 final_price:1.11985 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:26.995000 package:None PV:5.0000000000105516e-05 price:5.0000000000105516e-05 timestamp:2017-06-26 00:02:27.165000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None trade_maturity_date:28/06/2017 traded_px:1.1198 ccy:('EUR', 'USD')
 dt:10 final_price:1.11985 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_time

In [7]:
# run markouts using PricingContext
# run the markout calc
%load_ext autoreload
%autoreload 2
from mosaicsmartdata.core.fx import FXPricingContextGenerator
quotes_eur = quote_dict['EUR=']
quotes_jpy = quote_dict['JPY=']
print(len(quotes_jpy),len(quotes_eur))
quotes_all = op.merge_sorted([quotes_eur[:1000],quotes_jpy[:1000]], lambda x: x.timestamp) > []
             #|\op.map(lambda x: (x.sym, x.timestamp)) > 
run(quotes_all)

my_quote = quotes_all.sink[100]
my_instr_2 = sym_to_instrument()('EURJPY=', my_quote.timestamp.date())
my_instr_2.notionals = (100, -100*124.594511) # just picked a random quote
print(my_instr_2) 
print('*****')
my_trade_2 = FXForwardTrade(my_instr_2, 
                          trade_id = '12346',
                        timestamp = my_quote.timestamp)
print(my_trade_2.timestamp)


553108 591602
<class 'mosaicsmartdata.core.instrument.FXForward'> :  static = Instrument static singleton facade, notionals = (100, -12459.4511), sym = EURJPY=, venue = None, settle_date = 2017-06-28, maturity_date = 2017-06-28, isDeliverable = True, pip_size = 0.0001, holidayCities = None, date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x00000176C8AF5D30>, ccy = ('EUR', 'JPY'), tenor = SPOT
*****
2017-06-26 00:00:53.188000


In [8]:
# forward-looking markouts using PricingContext
trades = [my_trade]
quotes_2 = quotes_all.sink
#print([quote.sym for quote in quotes_2][:100])

prices =  quotes_2 | op.map(FXPricingContextGenerator()) 
#stream = prices | op.map(lambda x: x.timestamp) > print
stream = op.merge_sorted([trades,prices], lambda x:x.timestamp)\
        | op.flat_map(MarkoutCalculator([-10,0, 10, 100])) | op.map(lambda x:str(x))\
        > []
run(stream)       

stream.sink

{'EUR': 0.8928970043305505} {'EUR': {}} {}
{'EUR': 0.8928970043305505, 'JPY': 111.24000000000001} {'EUR': {}, 'JPY': {}} {}


[" dt:-10 final_price:nan hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:16.995000 package:None PV:nan price:nan timestamp:2017-06-26 00:02:26.995000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None trade_maturity_date:28/06/2017 traded_px:1.1198 ccy:('EUR', 'USD')"]

In [9]:
# extract just the first quote from every stream
tiny_quote_dict = {key:[quotes[0]] for key, quotes in quote_dict.items()}
print([(key,quotes[0].bid, quotes[0].instrument.ccy, quotes[0].instrument.sp) for key, quotes in tiny_quote_dict.items()])
with open('../resources/fx/data.pickle', 'wb') as f:
        pickle.dump(tiny_quote_dict, f)


AttributeError: 'OIS' object has no attribute 'sp'

In [ ]:
with open('../resources/fx/data.pickle', 'rb') as f:
        tiny_quote_dict = pickle.load(f)
        
tiny_quotes_list = [quotes for key, quotes in tiny_quote_dict.items()]
stream = op.merge_sorted(tiny_quotes_list, lambda x: x.timestamp) |\
            op.map(FXPricingContextGenerator()) > print
run(stream)

In [ ]:
a = '0B'
a[-1].lower() =='b'
